In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('Iris.csv')
df

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...,...
145,146,6.7,3.0,5.2,2.3,Iris-virginica
146,147,6.3,2.5,5.0,1.9,Iris-virginica
147,148,6.5,3.0,5.2,2.0,Iris-virginica
148,149,6.2,3.4,5.4,2.3,Iris-virginica


In [7]:
from sklearn.model_selection import train_test_split

X = df.iloc[:, 1:-1]
y = df.iloc[:, -1]
#one hot encoding
y = pd.get_dummies(y).values
X_train, X_test, y_train, y_test  = train_test_split(X, y, random_state = 12, train_size = 0.8)

# print("THE TRAINING SET ARE\n")
# print("X_train") 
# print(X_train) 
# print("\ny_train")
# print(y_train)
# print("\n\n\n")
# print("THE TESTING SET ARE\n")
# print("X_test")
# print(X_test)
# print("\ny_test")
# print(y_test)

X_train = np.array(X_train.iloc[:, 0:4])
X_test = np.array(X_test.iloc[:, 0:4])


input_size = 4
hidden_sizes = [6,6,3]  
output_size = 3

np.random.seed(42)
weights = []
biases = []
for i in range(len(hidden_sizes) + 1):
    if i == 0: 
        weights.append(np.random.randn(input_size, hidden_sizes[i]))
        biases.append(np.zeros((1, hidden_sizes[i])))
    elif i == len(hidden_sizes):
        weights.append(np.random.randn(hidden_sizes[i-1], output_size))
        biases.append(np.zeros((1, output_size)))
    else: 
        weights.append(np.random.randn(hidden_sizes[i-1], hidden_sizes[i]))
        biases.append(np.zeros((1, hidden_sizes[i])))

def sigmoid(x):
    return 1 / (1 + np.exp(-x))
def sigmoid_derivative(x):
    return x * (1 - x)

def tanh(x):
    return np.tanh(x)
def tanh_derivative(x):
    return 1 - np.square(np.tanh(x))


def activation_function(x):
    return sigmoid(x)
def activation_function_derivative(x):
    return sigmoid_derivative(x)

learning_rate = 0.01
num_epochs = 10000

for epoch in range(num_epochs):
    # Forward pass
    hidden_layer_outputs = [X_train]
    for i in range(len(hidden_sizes)):
        hidden_layer_input = np.dot(hidden_layer_outputs[i], weights[i]) + biases[i]
        hidden_layer_output = activation_function(hidden_layer_input)
        hidden_layer_outputs.append(hidden_layer_output)
    output_layer_input = np.dot(hidden_layer_outputs[-1], weights[-1]) + biases[-1]
    output_layer_output = activation_function(output_layer_input)

    # Backward pass
    error = y_train - output_layer_output
    d_output = error * activation_function_derivative(output_layer_output)

    deltas = [d_output]
    for i in reversed(range(len(hidden_sizes))):
        error_hidden = deltas[-1].dot(weights[i+1].T)
        d_hidden = error_hidden * activation_function_derivative(hidden_layer_outputs[i+1])
        deltas.append(d_hidden)
    deltas.reverse()

    for i in range(len(weights)):
        weights[i] += hidden_layer_outputs[i].T.dot(deltas[i]) * learning_rate
        biases[i] += np.sum(deltas[i], axis=0, keepdims=True) * learning_rate

# Test the neural network
hidden_layer_outputs = [X_test]
for i in range(len(hidden_sizes)):
    hidden_layer_input = np.dot(hidden_layer_outputs[i], weights[i]) + biases[i]
    hidden_layer_output = activation_function(hidden_layer_input)
    hidden_layer_outputs.append(hidden_layer_output)
output_layer_input = np.dot(hidden_layer_outputs[-1], weights[-1]) + biases[-1]
output_layer_output = activation_function(output_layer_input)
#output_layer_output = softmax(output_layer_input)

predictions = np.argmax(output_layer_output, axis=1)
y_test_=np.argmax(y_test,axis=1)
print("Predicted Values :", predictions)
print("   Y_Test Values :", y_test_)

accuracy = np.mean(predictions == y_test_) * 100
print("Accuracy: {}%".format(accuracy))

Predicted Values : [0 2 0 1 2 2 2 0 2 0 1 0 0 0 1 2 2 1 0 2 0 1 2 1 0 2 1 1 0 0]
   Y_Test Values : [0 2 0 1 2 2 2 0 2 0 1 0 0 0 1 2 2 1 0 1 0 1 2 1 0 2 1 1 0 0]
Accuracy: 96.66666666666667%


### ReLu

In [54]:
import numpy as np
import pandas as pd

df = pd.read_csv('Iris.csv')

from sklearn.model_selection import train_test_split
X = df.iloc[:, 1:-1]
y = df.iloc[:, -1]
#one hot encoding
y = pd.get_dummies(y).values
X_train, X_test, y_train, y_test  = train_test_split(X, y, random_state = 12, train_size = 0.8)
X_train = np.array(X_train.iloc[:, 0:4])
X_test = np.array(X_test.iloc[:, 0:4])
class ReLU:
    def forward(self, x):
        return np.maximum(0, x)
    def backward(self, x):
        return (x > 0).astype(int)

class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size):
        self.weights1 = np.random.randn(input_size, hidden_size) * np.sqrt(2 / input_size)
        self.bias1 = np.zeros((1, hidden_size))
        self.weights2 = np.random.randn(hidden_size, output_size) * np.sqrt(2 / hidden_size)
        self.bias2 = np.zeros((1, output_size))

    def forward(self, x):
        self.z1 = np.dot(x, self.weights1) + self.bias1
        self.a1 = ReLU().forward(self.z1)
        self.z2 = np.dot(self.a1, self.weights2) + self.bias2
        self.y_hat = self.z2
        return self.y_hat

    def backward(self, x, y, y_hat):
        m = y.shape[0]
        self.dz2 = (y_hat - y) / m
        self.dw2 = np.dot(self.a1.T, self.dz2)
        self.db2 = np.sum(self.dz2, axis=0, keepdims=True)
        self.da1 = np.dot(self.dz2, self.weights2.T)
        self.dz1 = self.da1 * ReLU().backward(self.z1)
        self.dw1 = np.dot(x.T, self.dz1)
        self.db1 = np.sum(self.dz1, axis=0, keepdims=True)

    def update(self, learning_rate):
        self.weights1 -= learning_rate * self.dw1
        self.bias1 -= learning_rate * self.db1
        self.weights2 -= learning_rate * self.dw2
        self.bias2 -= learning_rate * self.db2

    def train(self, x_train, y_train, epochs, learning_rate):
        for i in range(epochs):
            y_hat = self.forward(x_train)
            self.backward(x_train, y_train, y_hat)
            self.update(learning_rate)

    def predict(self, x):
        y_hat = self.forward(x)
        return np.argmax(y_hat, axis=1)


ann = NeuralNetwork(4,10,3)
ann.train(X_train, y_train, 10000, 0.01)
predictions = ann.predict(X_test)
y_test=np.argmax(y_test,axis=1)
print(predictions)
print(y_test)
accuracy = np.mean(predictions == y_test) * 100
print("Accuracy: {}%".format(accuracy))

[0 2 0 1 2 2 2 0 2 0 1 0 0 0 1 2 2 1 0 1 0 1 2 1 0 2 1 1 0 0]
[0 2 0 1 2 2 2 0 2 0 1 0 0 0 1 2 2 1 0 1 0 1 2 1 0 2 1 1 0 0]
Accuracy: 100.0%
